In [12]:
import numpy as np
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import great_tables as tg
import altair as alt

In [13]:
df_path = r'../DATASETS/Mall_Customers.csv'

In [14]:
df = pl.read_csv(df_path)

In [17]:
df

CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
i64,str,i64,i64,i64
1,"""Male""",19,15,39
2,"""Male""",21,15,81
3,"""Female""",20,16,6
4,"""Female""",23,16,77
5,"""Female""",31,17,40
…,…,…,…,…
196,"""Female""",35,120,79
197,"""Female""",45,126,28
198,"""Male""",32,126,74


In [22]:
X = df.select(['Annual Income (k$)', 'Spending Score (1-100)'])

In [27]:
fig = px.scatter(
    X,
    x='Annual Income (k$)',
    y='Spending Score (1-100)'
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [28]:
from sklearn.cluster import KMeans

In [31]:
wcss = []

for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=42, verbose=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

C:\Users\Zygim\anaconda3\envs\Analytics__2024_12_29\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

C:\Users\Zygim\anaconda3\envs\Analytics__2024_12_29\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

C:\Users\Zygim\anaconda3\envs\Analytics__2024_12_29\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

C:\Users\Zygim\anaconda3\envs\Analytics__2024_12_29\Lib\site-packages\sklearn\cluster\_kmeans.py:142

In [35]:
fig = px.line(
    x=range(1, 11),
    y=wcss
).update_layout(
    width=1000,
    height=800,
    title='The Elbow Method',
    xaxis_title='Number of clusters',
    yaxis_title='WCSS'
)

fig.show()

In [37]:
kmeans_model = KMeans(n_clusters=5, init='k-means++', random_state=42)
y_kmeans = kmeans_model.fit_predict(X)

C:\Users\Zygim\anaconda3\envs\Analytics__2024_12_29\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [39]:
pl.DataFrame(
    {
        'Annual Income (k$)': X['Annual Income (k$)'],
        'Spending Score (1-100)': X['Spending Score (1-100)'],
        'Cluster': y_kmeans
    }
)

Annual Income (k$),Spending Score (1-100),Cluster
i64,i64,i32
15,39,4
15,81,2
16,6,4
16,77,2
17,40,4
…,…,…
120,79,1
126,28,3
126,74,1


In [58]:
# X = X.to_pandas()

fig = px.scatter(
    x=X.loc[y_kmeans == 0, 'Annual Income (k$)'],
    y=X.loc[y_kmeans == 0, 'Spending Score (1-100)'],
    title='Clusters of Customers'
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [59]:
fig = px.scatter(
    x=X.loc[y_kmeans == 1, 'Annual Income (k$)'],
    y=X.loc[y_kmeans == 1, 'Spending Score (1-100)'],
    title='Clusters of Customers',
    color_discrete_sequence=["red"]
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [60]:
fig = px.scatter(
    x=X.loc[y_kmeans == 2, 'Annual Income (k$)'],
    y=X.loc[y_kmeans == 2, 'Spending Score (1-100)'],
    title='Clusters of Customers',
    color_discrete_sequence=["red"]
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [61]:
fig = px.scatter(
    x=X.loc[y_kmeans == 3, 'Annual Income (k$)'],
    y=X.loc[y_kmeans == 3, 'Spending Score (1-100)'],
    title='Clusters of Customers',
    color_discrete_sequence=["green"]
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [62]:
fig = px.scatter(
    x=X.loc[y_kmeans == 4, 'Annual Income (k$)'],
    y=X.loc[y_kmeans == 4, 'Spending Score (1-100)'],
    title='Clusters of Customers',
    color_discrete_sequence=["yellow"]
).update_layout(
    width=1000,
    height=800,
)

fig.show()

In [81]:
combined_df = X # need to do to pandas
combined_df['Cluster'] = y_kmeans

cluster_colors = ["blue", "red", "green", "yellow", "purple"]

fig = px.scatter(
    combined_df,
    x='Annual Income (k$)',
    y='Spending Score (1-100)',
    color='Cluster',
    color_discrete_sequence=cluster_colors,
    title="Clusters of Customers with Centers"
)

fig.add_trace(
    go.Scatter(
        x=kmeans_model.cluster_centers_[:, 0],
        y=kmeans_model.cluster_centers_[:, 1],
        mode='markers',
        marker=dict(
            size=15,
            color='black',
            symbol='x'
        ),
        name="Cluster Centers"
    )
)

fig.update_layout(
    width=1000,
    height=800
)

fig.show()